In [700]:
import warnings

# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')

# 1. 데이터 불러오기

In [701]:
import pandas as pd
import os

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', None)

PATH = '../data/'

file_list= os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)
data = ['living_popul', 'resident_popul', 'area', 'store', 'working_popul', 'facilitie', 'estimated_sales']

for name, file in zip(data, csv_list):
    globals()[name] = pd.read_csv(PATH + file, encoding = "ansi")

store = store.drop("Unnamed: 0", axis = 1)
estimated_sales = estimated_sales.drop("Unnamed: 0", axis = 1)

# 2. 데이터 전처리

## 2-0. 시점 통일 / 관광특구 제거

In [702]:
# 시점 통일
years = [20223, 20224, 20231, 20232]

living_popul = living_popul[living_popul['기준_년분기_코드'].isin(years)]
resident_popul = resident_popul[resident_popul['기준_년분기_코드'].isin(years)]
store = store[store['기준_년분기_코드'].isin(years)]
working_popul = working_popul[working_popul['기준_년분기_코드'].isin(years)]
facilitie = facilitie[facilitie['기준_년분기_코드'].isin(years)]
estimated_sales = estimated_sales[estimated_sales['기준_년분기_코드'].isin(years)]

# 관광특구 제거
living_popul = living_popul[living_popul['상권_구분_코드_명']!='관광특구']
resident_popul = resident_popul[resident_popul['상권_구분_코드_명']!='관광특구']
store = store[store['상권_구분_코드_명']!='관광특구']
working_popul = working_popul[working_popul['상권_구분_코드_명']!='관광특구']
facilitie = facilitie[facilitie['상권_구분_코드_명']!='관광특구']
estimated_sales = estimated_sales[estimated_sales['상권_구분_코드_명']!='관광특구']

## 2-1. 길단위인구 전처리

In [703]:
living_popul.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6573 entries, 6 to 6596
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   기준_년분기_코드         6573 non-null   int64 
 1   상권_구분_코드_명        6573 non-null   object
 2   상권_코드_명           6573 non-null   object
 3   총_유동인구_수          6573 non-null   int64 
 4   남성_유동인구_수         6573 non-null   int64 
 5   여성_유동인구_수         6573 non-null   int64 
 6   연령대_10_유동인구_수     6573 non-null   int64 
 7   연령대_20_유동인구_수     6573 non-null   int64 
 8   연령대_30_유동인구_수     6573 non-null   int64 
 9   연령대_40_유동인구_수     6573 non-null   int64 
 10  연령대_50_유동인구_수     6573 non-null   int64 
 11  연령대_60_이상_유동인구_수  6573 non-null   int64 
 12  시간대_00_06_유동인구_수  6573 non-null   int64 
 13  시간대_06_11_유동인구_수  6573 non-null   int64 
 14  시간대_11_14_유동인구_수  6573 non-null   int64 
 15  시간대_14_17_유동인구_수  6573 non-null   int64 
 16  시간대_17_21_유동인구_수  6573 non-null   int64 
 17  시간대_21_24_유동인구_수  6

In [704]:
living_popul.head()

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,시간대_00_06_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_24_유동인구_수,월요일_유동인구_수,화요일_유동인구_수,수요일_유동인구_수,목요일_유동인구_수,금요일_유동인구_수,토요일_유동인구_수,일요일_유동인구_수
6,20232,전통시장,"평화시장(남평화시장, 제일평화시장, 신평화패션타운)",61694,28879,32813,2470,9157,15006,13271,10280,11510,14369,12350,9136,8399,9354,8085,10022,10201,10244,10277,9029,5631,6289
7,20232,전통시장,고덕 골목형상점가,139522,64543,74979,31062,11733,21297,26684,18241,30506,33901,32903,17319,16182,22439,16779,19919,19959,19798,20017,19971,19959,19896
8,20232,전통시장,명일전통시장,290772,127429,163344,50790,31745,39532,47327,43358,78023,58298,56603,38509,39955,60687,36723,41182,41139,41623,40730,41754,41800,42545
9,20232,전통시장,길동복조리시장,528866,237050,291815,79440,57909,78798,81865,82159,148695,153914,106485,58851,57556,82890,69170,75277,75055,73855,74509,75881,77237,77054
10,20232,전통시장,둔촌역전통시장,310288,151555,158734,40335,39059,40908,50746,52919,86320,87311,62658,33947,34930,50959,40484,44098,43906,44155,43838,44533,45236,44522


### 2-1-1 변수 처리

In [705]:
# 파생변수 생성
living_popul['시간대_21_06_유동인구_수'] = living_popul['시간대_00_06_유동인구_수'] + living_popul['시간대_21_24_유동인구_수']

living_popul['주중_유동인구_수'] = living_popul['월요일_유동인구_수'] + living_popul['화요일_유동인구_수'] + living_popul['수요일_유동인구_수'] + living_popul['목요일_유동인구_수'] + living_popul['금요일_유동인구_수']
living_popul['주말_유동인구_수'] = living_popul['토요일_유동인구_수'] + living_popul['일요일_유동인구_수']

living_popul['연령대_30_40_유동인구_수'] = living_popul['연령대_30_유동인구_수'] + living_popul['연령대_40_유동인구_수']
living_popul['연령대_50_60_유동인구_수'] = living_popul['연령대_50_유동인구_수'] + living_popul['연령대_60_이상_유동인구_수']

# 불필요한 변수 제거
living_popul = living_popul.drop(['시간대_00_06_유동인구_수', '시간대_21_24_유동인구_수',
                                  '월요일_유동인구_수', '화요일_유동인구_수', '수요일_유동인구_수', '목요일_유동인구_수', '금요일_유동인구_수', '토요일_유동인구_수', '일요일_유동인구_수', 
                                  '연령대_30_유동인구_수','연령대_40_유동인구_수','연령대_50_유동인구_수','연령대_60_이상_유동인구_수'], axis = 1)

### 2-1-2. 결측치/이상치/중복

In [706]:
# 결측값 확인
living_popul.isnull().sum()

기준_년분기_코드           0
상권_구분_코드_명          0
상권_코드_명             0
총_유동인구_수            0
남성_유동인구_수           0
여성_유동인구_수           0
연령대_10_유동인구_수       0
연령대_20_유동인구_수       0
시간대_06_11_유동인구_수    0
시간대_11_14_유동인구_수    0
시간대_14_17_유동인구_수    0
시간대_17_21_유동인구_수    0
시간대_21_06_유동인구_수    0
주중_유동인구_수           0
주말_유동인구_수           0
연령대_30_40_유동인구_수    0
연령대_50_60_유동인구_수    0
dtype: int64

In [707]:
# 중복 확인
living_popul[living_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수


In [708]:
# 통계량 확인
living_popul.describe()

,기준_년분기_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수
count,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000
mean,20227.499,819172.761,390059.275,429113.498,105750.179,146463.772,166764.988,104697.384,105734.445,141765.368,300210.649,590223.923,228948.919,276078.394,290880.488
std,4.032,860738.310,415405.204,448793.459,112986.973,211285.817,172023.931,119372.114,122503.528,158734.038,318552.356,625430.046,240185.552,311897.390,293363.299
min,20223.000,12.000,12.000,0.000,0.000,0.000,4.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,221071.000,106737.000,113709.000,25585.000,32148.000,45786.000,28449.000,28568.000,38153.000,75518.000,158928.000,60054.000,74201.000,79250.000
50%,20224.000,566885.000,269133.000,296501.000,71313.000,82738.000,116615.000,71878.000,71921.000,95899.000,205337.000,405241.000,158567.000,184591.000,205254.000
75%,20231.000,1130788.000,534060.000,588669.000,146355.000,182991.000,233054.000,139191.000,139491.000,193507.000,414441.000,822348.000,311102.000,369148.000,409490.000
max,20232.000,7942915.000,4000287.000,4032096.000,829062.000,3487417.000,1529921.000,1429461.000,1491003.000,1793125.000,2942275.000,6236830.000,2124591.000,3433420.000,2463853.000


### 2-1-3. 분기별 데이터 연도 기준 변환

In [709]:
living_popul_year = living_popul.drop('기준_년분기_코드', axis = 1)\
                                .groupby(['상권_구분_코드_명','상권_코드_명'],as_index = False)\
                                .mean()

In [710]:
living_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1644 entries, 0 to 1643
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   상권_구분_코드_명        1644 non-null   object 
 1   상권_코드_명           1644 non-null   object 
 2   총_유동인구_수          1644 non-null   float64
 3   남성_유동인구_수         1644 non-null   float64
 4   여성_유동인구_수         1644 non-null   float64
 5   연령대_10_유동인구_수     1644 non-null   float64
 6   연령대_20_유동인구_수     1644 non-null   float64
 7   시간대_06_11_유동인구_수  1644 non-null   float64
 8   시간대_11_14_유동인구_수  1644 non-null   float64
 9   시간대_14_17_유동인구_수  1644 non-null   float64
 10  시간대_17_21_유동인구_수  1644 non-null   float64
 11  시간대_21_06_유동인구_수  1644 non-null   float64
 12  주중_유동인구_수         1644 non-null   float64
 13  주말_유동인구_수         1644 non-null   float64
 14  연령대_30_40_유동인구_수  1644 non-null   float64
 15  연령대_50_60_유동인구_수  1644 non-null   float64
dtypes: float64(14), object(2)
memory usage: 20

## 2-2. 상주인구 전처리

In [711]:
resident_popul.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6488 entries, 15015 to 24481
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   기준_년분기_코드           6488 non-null   int64 
 1   상권_구분_코드_명          6488 non-null   object
 2   상권_코드_명             6488 non-null   object
 3   총_상주인구_수            6488 non-null   int64 
 4   남성_상주인구_수           6488 non-null   int64 
 5   여성_상주인구_수           6488 non-null   int64 
 6   연령대_10_상주인구_수       6488 non-null   int64 
 7   연령대_20_상주인구_수       6488 non-null   int64 
 8   연령대_30_상주인구_수       6488 non-null   int64 
 9   연령대_40_상주인구_수       6488 non-null   int64 
 10  연령대_50_상주인구_수       6488 non-null   int64 
 11  연령대_60_이상_상주인구_수    6488 non-null   int64 
 12  남성연령대_10_상주인구_수     6488 non-null   int64 
 13  남성연령대_20_상주인구_수     6488 non-null   int64 
 14  남성연령대_30_상주인구_수     6488 non-null   int64 
 15  남성연령대_40_상주인구_수     6488 non-null   int64 
 16  남성연령대_50_상주인구_수     6488

In [712]:
resident_popul.head()

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_상주인구_수,연령대_40_상주인구_수,연령대_50_상주인구_수,연령대_60_이상_상주인구_수,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수
15015,20223,골목상권,이북5도청사,1020,466,554,160,142,94,160,194,270,85,64,44,68,84,121,75,78,50,92,110,149,401,0,401
15016,20223,골목상권,독립문역 1번,629,309,320,79,57,59,78,91,265,39,29,30,45,51,115,40,28,29,33,40,150,343,0,343
15017,20223,골목상권,세검정초등학교,3212,1505,1707,603,399,372,551,533,754,285,203,179,256,234,348,318,196,193,295,299,406,1270,0,1270
15018,20223,골목상권,대신고등학교,471,221,250,57,61,47,69,77,160,30,34,24,33,33,67,27,27,23,36,44,93,244,0,244
15019,20223,골목상권,세검정,2153,1044,1109,356,294,250,346,374,533,205,153,119,152,173,242,151,141,131,194,201,291,878,0,878


### 2-2-1. 변수 처리

In [713]:
# 파생변수 생성
resident_popul['연령대_30_40_상주인구_수'] = resident_popul['연령대_30_상주인구_수']+resident_popul['연령대_40_상주인구_수']
resident_popul['연령대_50_60_상주인구_수'] = resident_popul['연령대_50_상주인구_수']+resident_popul['연령대_60_이상_상주인구_수']

# 불필요한 변수 제거
resident_popul = resident_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명',
                                 '총_상주인구_수', '남성_상주인구_수','여성_상주인구_수', 
                                 '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_40_상주인구_수', '연령대_50_60_상주인구_수']]

### 2-2-2. 결측치/이상치/중복

In [714]:
# 결측값 확인
resident_popul.isnull().sum()

기준_년분기_코드           0
상권_구분_코드_명          0
상권_코드_명             0
총_상주인구_수            0
남성_상주인구_수           0
여성_상주인구_수           0
연령대_10_상주인구_수       0
연령대_20_상주인구_수       0
연령대_30_40_상주인구_수    0
연령대_50_60_상주인구_수    0
dtype: int64

In [715]:
# 중복 확인
resident_popul[resident_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수


In [716]:
# 통계량 확인
resident_popul.describe()

,기준_년분기_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수
count,6488.000,6488.000,6488.000,6488.000,6488.000,6488.000,6488.000,6488.000
mean,20227.495,2359.471,1165.882,1193.589,230.282,300.194,789.012,1039.983
std,4.032,2348.110,1159.129,1194.338,262.202,339.598,833.507,1031.546
min,20223.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,657.000,326.750,330.750,53.000,70.000,207.000,280.000
50%,20224.000,1614.000,809.000,817.000,147.000,195.000,528.000,720.000
75%,20231.000,3305.000,1639.000,1677.000,318.000,413.250,1083.000,1464.250
max,20232.000,21341.000,10459.000,10882.000,3078.000,4457.000,7626.000,8342.000


### 2-2-3. 분기별 데이터 연도 기준 변환

In [717]:
resident_popul_year = resident_popul[resident_popul['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'],axis=1)

In [718]:
resident_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1619 entries, 19887 to 24481
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   상권_구분_코드_명        1619 non-null   object
 1   상권_코드_명           1619 non-null   object
 2   총_상주인구_수          1619 non-null   int64 
 3   남성_상주인구_수         1619 non-null   int64 
 4   여성_상주인구_수         1619 non-null   int64 
 5   연령대_10_상주인구_수     1619 non-null   int64 
 6   연령대_20_상주인구_수     1619 non-null   int64 
 7   연령대_30_40_상주인구_수  1619 non-null   int64 
 8   연령대_50_60_상주인구_수  1619 non-null   int64 
dtypes: int64(7), object(2)
memory usage: 126.5+ KB


## 2-3. 점포 전처리

### 2-3-1. 변수 처리

In [719]:
# 불필요한 변수 삭제
store.drop(columns = ['상권_구분_코드', '상권_코드'], inplace = True)

### 2-3-1. 결측치/이상치/중복

In [720]:
# 결측값 확인
store.isnull().sum()

기준_년분기_코드      0
상권_구분_코드_명     0
상권_코드_명        0
서비스_업종_코드      0
서비스_업종_코드_명    0
점포_수           0
유사_업종_점포_수     0
개업_율           0
개업_점포_수        0
폐업_률           0
폐업_점포_수        0
프랜차이즈_점포_수     0
dtype: int64

In [721]:
# 중복 확인
store[store.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수


In [722]:
# 통계량 확인
store.describe()

,기준_년분기_코드,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
count,302726.000,302726.000,302726.000,302726.000,302726.000,302726.000,302726.000,302726.000
mean,20227.497,5.692,6.126,2.296,0.134,2.482,0.171,0.434
std,4.031,25.823,26.281,10.924,0.584,11.827,0.784,2.047
min,20223.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000
50%,20224.000,2.000,2.000,0.000,0.000,0.000,0.000,0.000
75%,20231.000,4.000,5.000,0.000,0.000,0.000,0.000,0.000
max,20232.000,2190.000,2191.000,200.000,28.000,400.000,71.000,123.000


### 2-3-2. 분기별 데이터 연도 기준 변환

In [723]:
store_year = store[store['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드', '서비스_업종_코드'], axis = 1)\
                                                    .groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명'], as_index = False)\
                                                    .sum()

In [724]:
store_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75589 entries, 0 to 75588
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   상권_구분_코드_명   75589 non-null  object
 1   상권_코드_명      75589 non-null  object
 2   서비스_업종_코드_명  75589 non-null  object
 3   점포_수         75589 non-null  int64 
 4   유사_업종_점포_수   75589 non-null  int64 
 5   개업_율         75589 non-null  int64 
 6   개업_점포_수      75589 non-null  int64 
 7   폐업_률         75589 non-null  int64 
 8   폐업_점포_수      75589 non-null  int64 
 9   프랜차이즈_점포_수   75589 non-null  int64 
dtypes: int64(7), object(3)
memory usage: 5.8+ MB


## 2-4. 직장인구 전처리

### 2-4-1. 변수 처리

In [725]:
working_popul = working_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '총_직장_인구_수']]

### 2-4-2. 결측치/이상치/중복

In [726]:
# 결측값 확인
working_popul.isnull().sum()

기준_년분기_코드     0
상권_구분_코드_명    0
상권_코드_명       0
총_직장_인구_수     0
dtype: int64

In [727]:
# 중복 확인
working_popul[working_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_직장_인구_수


In [728]:
# 통계량 확인
working_popul.describe()

,기준_년분기_코드,총_직장_인구_수
count,6468.000,6468.000
mean,20227.500,1660.112
std,4.031,5611.955
min,20223.000,1.000
25%,20223.750,105.000
50%,20227.500,315.000
75%,20231.250,876.000
max,20232.000,81127.000


### 2-4-3. 분기별 데이터 연도 기준 변환

In [729]:
working_popul_year = working_popul[working_popul['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'],axis=1)

In [730]:
working_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1617 entries, 11546 to 25988
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   상권_구분_코드_명  1617 non-null   object
 1   상권_코드_명     1617 non-null   object
 2   총_직장_인구_수   1617 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 50.5+ KB


## 2-5. 집객시설 전처리

### 2-5-1. 변수 처리

In [731]:
# 파생변수 생성
facilitie['초중고_수'] = facilitie['초등학교_수']+facilitie['중학교_수']+facilitie['고등학교_수']

facilitie['교통시설수'] = facilitie['버스_정거장_수'] + facilitie['지하철_역_수']
 
facilitie['집객시설수'] = facilitie['집객시설_수']-facilitie['교통시설수']

# 불필요한 변수 제거
facilitie = facilitie.drop(['집객시설_수', '철도_역_수', '초등학교_수', '중학교_수', '고등학교_수', '버스_정거장_수', '지하철_역_수'], axis=1)
facilitie = facilitie[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '집객시설수', '교통시설수' ]]

### 2-5-2. 결측치/이상치/중복

In [732]:
# 결측값 확인
facilitie.isnull().sum()

기준_년분기_코드        0
상권_구분_코드_명       0
상권_코드_명          0
집객시설수         5556
교통시설수         5556
dtype: int64

In [733]:
# 결측값 0으로 대체
facilitie = facilitie.fillna(0)

In [734]:
# 중복 확인
facilitie[facilitie.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,집객시설수,교통시설수


In [735]:
# 통계량 확인
facilitie.describe()

,기준_년분기_코드,집객시설수,교통시설수
count,6288.000,6288.000,6288.000
mean,20227.500,6.245,1.243
std,4.031,23.680,4.459
min,20223.000,0.000,0.000
25%,20223.750,0.000,0.000
50%,20227.500,0.000,0.000
75%,20231.250,0.000,0.000
max,20232.000,351.000,87.000


### 2-5-3. 분기별 데이터 연도 기준 변환

In [736]:
facilitie_year = facilitie[facilitie['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'], axis = 1)

In [737]:
facilitie_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1572 entries, 0 to 17269
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   상권_구분_코드_명  1572 non-null   object 
 1   상권_코드_명     1572 non-null   object 
 2   집객시설수       1572 non-null   float64
 3   교통시설수       1572 non-null   float64
dtypes: float64(2), object(2)
memory usage: 61.4+ KB


## 2-6. 추정매출 전처리

### 2-6-1. 변수 처리

In [738]:
# 파생변수 생성
estimated_sales['시간대_21~06_매출_금액'] = estimated_sales['시간대_00~06_매출_금액'] + estimated_sales['시간대_21~24_매출_금액']
estimated_sales['시간대_21~06_매출_건수'] = estimated_sales['시간대_건수~06_매출_건수'] + estimated_sales['시간대_건수~24_매출_건수']
estimated_sales['연령대_30_40_매출_금액'] = estimated_sales['연령대_30_매출_금액'] + estimated_sales['연령대_40_매출_금액']
estimated_sales['연령대_50_60_매출_금액'] = estimated_sales['연령대_50_매출_금액'] + estimated_sales['연령대_60_이상_매출_금액']
estimated_sales['연령대_30_40_매출_건수'] = estimated_sales['연령대_30_매출_건수'] + estimated_sales['연령대_40_매출_건수']
estimated_sales['연령대_50_60_매출_건수'] = estimated_sales['연령대_50_매출_건수'] + estimated_sales['연령대_60_이상_매출_건수']

# 불필요한 변수 제거
estimated_sales = estimated_sales.drop(columns = ['서비스_업종_코드'],
                                       axis = 1)

estimated_sales = estimated_sales.drop(columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액',
                                                  '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수'],
                                       axis = 1)
estimated_sales = estimated_sales.drop(columns = ['시간대_00~06_매출_금액', '시간대_21~24_매출_금액', '시간대_건수~06_매출_건수', '시간대_건수~24_매출_건수',
                                                 '연령대_30_매출_금액','연령대_40_매출_금액','연령대_50_매출_금액','연령대_60_이상_매출_금액',
                                                 '연령대_30_매출_건수','연령대_40_매출_건수','연령대_50_매출_건수','연령대_60_이상_매출_건수'],
                                       axis = 1)

# 변수 이름 변경
estimated_sales = estimated_sales.rename(columns = {'시간대_건수~11_매출_건수' : '시간대_06~11_매출_건수',
                                                         '시간대_건수~14_매출_건수' : '시간대_11~14_매출_건수',
                                                         '시간대_건수~17_매출_건수' : '시간대_14~17_매출_건수',
                                                         '시간대_건수~21_매출_건수' : '시간대_17~21_매출_건수'})


def sales_tran(df):
    if df['당월_법인_매출_금액'] < 0:
        return df['당월_매출_금액'] - df['당월_법인_매출_금액']
    else :
        return df['당월_매출_금액']

estimated_sales['당월_개인_매출_금액'] = estimated_sales['남성_매출_금액'] + estimated_sales['여성_매출_금액']
estimated_sales['당월_법인_매출_금액'] =  estimated_sales['당월_매출_금액'] - estimated_sales['당월_개인_매출_금액']

estimated_sales['당월_매출_금액'] = estimated_sales.apply(lambda x : sales_tran(x) , axis=1)

estimated_sales.loc[estimated_sales['당월_법인_매출_금액']<0, '당월_법인_매출_금액'] = 0

### 2-6-2. 결측치/이상치/중복

In [739]:
# 결측값 확인
estimated_sales.isnull().sum()

기준_년분기_코드          0
상권_구분_코드           0
상권_구분_코드_명         0
상권_코드              0
상권_코드_명            0
서비스_업종_코드_명        0
당월_매출_금액           0
당월_매출_건수           0
주중_매출_금액           0
주말_매출_금액           0
시간대_06~11_매출_금액    0
시간대_11~14_매출_금액    0
시간대_14~17_매출_금액    0
시간대_17~21_매출_금액    0
남성_매출_금액           0
여성_매출_금액           0
연령대_10_매출_금액       0
연령대_20_매출_금액       0
주중_매출_건수           0
주말_매출_건수           0
시간대_06~11_매출_건수    0
시간대_11~14_매출_건수    0
시간대_14~17_매출_건수    0
시간대_17~21_매출_건수    0
남성_매출_건수           0
여성_매출_건수           0
연령대_10_매출_건수       0
연령대_20_매출_건수       0
시간대_21~06_매출_금액    0
시간대_21~06_매출_건수    0
연령대_30_40_매출_금액    0
연령대_50_60_매출_금액    0
연령대_30_40_매출_건수    0
연령대_50_60_매출_건수    0
당월_개인_매출_금액        0
당월_법인_매출_금액        0
dtype: int64

In [740]:
# 통계량 확인
estimated_sales.describe()

,기준_년분기_코드,상권_코드,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수,당월_개인_매출_금액,당월_법인_매출_금액
count,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000
mean,20227.506,3116851.531,1057530630.270,37378.921,800998140.292,256532440.674,112812620.583,270639855.442,263642589.533,289522260.989,482553877.888,454435667.863,5388979.464,125426918.046,28218.137,9160.785,5231.569,9703.087,7869.907,9959.027,18572.037,17223.529,559.116,7452.457,120913224.832,4615.330,406737845.752,399433563.022,15695.358,12088.637,936989545.752,120541084.519
std,4.031,7072.715,9247199458.539,158825.344,6375468667.307,3472245561.224,1170991051.609,2448384035.026,3090139741.176,2707954382.716,4423202984.139,3474196467.177,29279037.589,839832564.084,111071.872,54745.766,27854.478,43249.044,41592.007,44177.137,80157.083,76442.682,3521.526,39113.158,983837619.257,22925.971,3379770530.233,3754679997.219,67955.953,60479.077,7676380836.375,2380499063.105
min,20223.000,3110001.000,12.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20224.000,3110566.000,48000000.000,859.000,35761235.750,7349956.500,135702.250,5399863.500,7746083.500,11697304.250,19337881.500,17810972.500,0.000,1333739.750,646.000,152.000,6.000,107.000,171.000,224.000,386.000,337.000,0.000,42.000,0.000,0.000,15640213.000,15286881.750,321.000,294.000,43778347.250,0.000
50%,20231.000,3120001.000,184854645.500,4585.000,138333420.000,37661145.000,6136944.500,34222861.000,35608985.500,49693963.000,82942230.000,74623180.000,160833.000,12158489.500,3400.000,1018.000,165.000,967.000,966.000,1287.000,2253.000,1937.500,12.000,438.000,3981946.500,76.000,68591283.500,62915461.000,1756.000,1620.000,168098750.000,1516748.500
75%,20232.000,3120186.000,670335090.750,24990.750,508560976.750,146729693.250,47316932.250,149821606.500,139409257.750,182213664.750,307190696.500,282747046.500,2212020.500,62126169.750,18704.250,5709.750,1965.000,6277.000,5095.000,6880.000,11848.000,11338.000,170.000,3233.750,50244980.000,1468.000,254483601.000,235029535.000,9660.000,8417.250,610772422.500,23182793.500
max,20232.000,3130327.000,995677000000.000,12090273.000,669799000000.000,485357000000.000,107916000000.000,242249000000.000,326543000000.000,386343000000.000,537243000000.000,304162000000.000,1549259735.000,90669553834.000,5827863.000,6262410.000,1474148.000,2611826.000,3245770.000,4287338.000,6708993.000,4615812.000,329040.000,3087414.000,126314287024.000,1514884.000,354347000000.000,411544000000.000,4519958.000,5329793.000,841405000000.000,391742210636.000


In [741]:
# 중복 확인
estimated_sales[estimated_sales.duplicated()]

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수,당월_개인_매출_금액,당월_법인_매출_금액


### 2-6-3. 분기별 데이터 연도 기준 변환

In [742]:
estimated_sales_year = estimated_sales.drop(columns = ['기준_년분기_코드','상권_구분_코드','상권_코드'])\
                                      .groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명'])\
                                      .mean()\
                                      .reset_index()

In [743]:
estimated_sales_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22006 entries, 0 to 22005
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   상권_구분_코드_명       22006 non-null  object 
 1   상권_코드_명          22006 non-null  object 
 2   서비스_업종_코드_명      22006 non-null  object 
 3   당월_매출_금액         22006 non-null  float64
 4   당월_매출_건수         22006 non-null  float64
 5   주중_매출_금액         22006 non-null  float64
 6   주말_매출_금액         22006 non-null  float64
 7   시간대_06~11_매출_금액  22006 non-null  float64
 8   시간대_11~14_매출_금액  22006 non-null  float64
 9   시간대_14~17_매출_금액  22006 non-null  float64
 10  시간대_17~21_매출_금액  22006 non-null  float64
 11  남성_매출_금액         22006 non-null  float64
 12  여성_매출_금액         22006 non-null  float64
 13  연령대_10_매출_금액     22006 non-null  float64
 14  연령대_20_매출_금액     22006 non-null  float64
 15  주중_매출_건수         22006 non-null  float64
 16  주말_매출_건수         22006 non-null  float64
 17  시간대_06~11_매출

## 2-7. 데이터 병합

In [744]:
df = pd.merge(living_popul_year, resident_popul_year)
df = pd.merge(df, store_year)
df = pd.merge(df, working_popul_year)
df = pd.merge(df, facilitie_year)
df = pd.merge(df, estimated_sales_year)
df = pd.merge(df, area)

In [745]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21676 entries, 0 to 21675
Data columns (total 69 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   상권_구분_코드_명        21676 non-null  object 
 1   상권_코드_명           21676 non-null  object 
 2   총_유동인구_수          21676 non-null  float64
 3   남성_유동인구_수         21676 non-null  float64
 4   여성_유동인구_수         21676 non-null  float64
 5   연령대_10_유동인구_수     21676 non-null  float64
 6   연령대_20_유동인구_수     21676 non-null  float64
 7   시간대_06_11_유동인구_수  21676 non-null  float64
 8   시간대_11_14_유동인구_수  21676 non-null  float64
 9   시간대_14_17_유동인구_수  21676 non-null  float64
 10  시간대_17_21_유동인구_수  21676 non-null  float64
 11  시간대_21_06_유동인구_수  21676 non-null  float64
 12  주중_유동인구_수         21676 non-null  float64
 13  주말_유동인구_수         21676 non-null  float64
 14  연령대_30_40_유동인구_수  21676 non-null  float64
 15  연령대_50_60_유동인구_수  21676 non-null  float64
 16  총_상주인구_수          21676 non-null  int64 

In [746]:
df.to_csv("model_df.csv", index = False)